In [1]:
using NamedArrays      # make sure you run Pkg.add("NamedArrays") first!

# import Stigler's data set
raw = readdlm("stigler.csv",',');
(m,n) = size(raw)

n_nutrients = 2:n      # columns containing nutrients
n_foods = 3:m          # rows containing food names

nutrients = raw[1,n_nutrients][:]   # the list of nutrients (convert to 1-D array)
foods = raw[n_foods,1][:]           # the list of foods (convert to 1-D array)

# lower[i] is the minimum daily requirement of nutrient i.
lower = Dict( zip(nutrients,raw[2,n_nutrients]) )

# data[f,i] is the amount of nutrient i contained in food f.
data = NamedArray( raw[n_foods,n_nutrients], (foods,nutrients), ("foods","nutrients") );


#----Dual Model----#
using JuMP
m = Model()

#Lambdas corresponding to RDA constraints
@defVar(m, lambda[nutrients] >= 0)

#Dual Constraints
@addConstraint(m, dualConst[f in foods], sum{data[f, n]*lambda[n], n in nutrients} <= 365)

#Maximize dot product of RDA and lambda matrices
@setObjective(m, Max, sum{lower[n]*lambda[n], n in nutrients})

solve(m)
#Display results
println("Lambda values:")
println()
println(getValue(lambda))

Lambda values:

lambda: 1 dimensions, 9 entries:
 [ Ascorbic Acid (mg)] = 0.052602893142533885
 [        Calcium (g)] = 11.5842654076576
 [    Calories (1000)] = 3.1992787637880906
 [          Iron (mg)] = 0.0
 [        Niacin (mg)] = 0.0
 [        Protein (g)] = 0.0
 [    Riboflavin (mg)] = 5.970681935235992
 [      Thiamine (mg)] = 0.0
 [Vitamin A (1000 IU)] = 0.1460849434297644
